In [78]:
# install packages
%pip install boto3
%pip install sagemaker
%pip install -U sagemaker
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [79]:
# cell 01
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import sagemaker
import torchmetrics
import torch
from sagemaker import get_execution_role


sagemaker_session = sagemaker.Session()

role = 'AmazonSageMaker-ExecutionRole-20220714T204241'
# role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [80]:
# cell 02
training_data_uri = 's3://multibert-8b87f872-a3fe-4a19-a016-e02402275450/training/multi_label_new.csv'
#test_data_uri = 's3://multibert-8b87f872-a3fe-4a19-a016-e02402275450/test'

In [81]:
from sagemaker.pytorch import PyTorch

In [88]:
# cell 05
pytorch_estimator = PyTorch('multibert.py',
                            instance_type='ml.g4dn.16xlarge',
                            instance_count=3,
                            role = role,
                            framework_version='1.11.0',
                            py_version='py38',
                            source_dir = 'code',
                            hyperparameters = {'epochs': 4, 'batch-size': 32, 'learning-rate': 2e-05})


In [89]:
# cell 07
pytorch_estimator.fit(training_data_uri)
#pytorch_estimator.fit({'train': training_data_uri,
#                       'test': test_data_uri})

2022-07-18 19:16:22 Starting - Starting the training job...
2022-07-18 19:16:46 Starting - Preparing the instances for trainingProfilerReport-1658171781: InProgress
.........
2022-07-18 19:18:26 Downloading - Downloading input data
2022-07-18 19:18:26 Training - Downloading the training image..................
2022-07-18 19:21:29 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-18 19:21:32,896 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-18 19:21:32,915 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-18 19:21:32,920 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-18 19:21:33,333 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requireme

UnexpectedStatusException: Error for Training job pytorch-training-2022-07-18-19-16-20-052: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "RuntimeError: CUDA error: device-side assert triggered
 CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
 For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
 
 During handling of the above exception, another exception occurred
 Traceback (most recent call last)
 File "multibert.py", line 147, in <module>
 multibert_classifier = model(train_df)
 File "multibert.py", line 76, in model
 train_model(LABEL_COLUMNS, warmup_steps, total_training_steps, data_module)
 File "/opt/ml/code/create_model.py", line 308, in train_model
 trainer.fit(model, data_module)
 File "/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 770, in fit
 self._call_and_handle_interrupt(
 File "/opt/conda/lib/python3.8/site-packages/pytorch_ligh

In [ ]:
# cell 09
predictor = pytorch_estimator.deploy(instance_type='ml.g4dn.xlarge',
                                     initial_instance_count=1)

In [ ]:
# cell 11
data = "I am a test."
response = predictor.predict(data)